In [1]:
# Learning LHS sampling

import matplotlib.pyplot as plt
import numpy as np


for i in range(5):
    for j in range(5):
        print(i*j)

0
0
0
0
0
0
1
2
3
4
0
2
4
6
8
0
3
6
9
12
0
4
8
12
16


In [27]:
# Next section is examples of creating LHS from list of variables. Take, for instance, an eight-layer RTS laminate.
# [pm phi_1 < T0_1 | T1_1 > n_1 , pm phi_2 < T0_2 | T1_2 > n_2]s such that there are 8 variables

# x = [phi1, t01, t11, n1, phi2, t02, t12, n2]

# as n_i is dependent on phi_i, normalise n_i to be between 0 and 1 and then, dependent on phi, define integer value

# total number of samples for an eight-layer RTS laminate (assuming symmetry and balance):
# n_phi=0 = [0, 1, ..., 10] therefore when phi = 0, number of variables for n = 11
# n_phi=90 [0, 1, ..., 18] therefore when phi = 90, number of variables for n = 18
# t0 = t1 = [0, 1, ..., 70] therefore number of variables = 71
# each cylinder (of eight layers) is defined by two RTS lamina: x = [phi1, t01, t11, n1, phi2, t02, t12, n2]

# total number of samples (phi = 0) + (phi = 0 and 90) + (phi = 90)
n_tot = (71**4*11**2) + 2*(71**4*11*19) + (71**4*19**2)
print(f'Total number of variables = {n_tot}')

Total number of variables = 22870512900


# Create LHS samples

In [6]:
# defining function to take LHS (n_samples-by-n_variables matrix of value 0 -> 1) into useable designs
def lhs_to_lamina(lhs_in, cyl, lam_type):

    # lhs_in = unity-normalised LHS
    # cyl = [radius, length, thickness]
    # lam_type = 1 (RTS) or lam_type = 0 (SF)
    
    # return = lamina = [[phi1, t01, t11, n1, phi2, t02, t12, n2]*n_var]

    # define sub-function that takes phi and returns n_max based on geometry
    def phi_to_n_max(cyl, phi):
        
        # cyl = [radius, length, thickness]
        # phi = 0 (axial) or = 90 (circumferential)
        
        # assume MSR (minimum steering radius) is 50, therefore 1 period is 100 mm (minimum)
        msr = 100
        
        if phi == 0:
            n_max = np.floor(cyl[1]//msr) + 1
        elif phi == 90:
            n_max = np.floor(2*np.pi*cyl[0]//msr) + 1
        else:
            SyntaxError('Phi is not defined as 0 or 90 degree')
            
        return n_max
    
    lamina = np.zeros((np.shape(lhs_in)))
    
    # if lam_type = 1 (i.e. RTS laminate)
    if lam_type:
    
        for i, l in enumerate(lhs_in):
            lamina[i, 0] = round(l[0])*90 # defining phi_1
            lamina[i, 1] = round(l[1]*70) # defining t0_1
            lamina[i, 2] = round(l[2]*70) # defining t1_1
            lamina[i, 3] = int(l[3]*phi_to_n_max(cyl, lamina[i,0])) # defining n_1
            lamina[i, 4] = round(l[4])*90 # defining phi_2
            lamina[i, 5] = round(l[5]*70) # defining t0_2
            lamina[i, 6] = round(l[6]*70) # defining t1_2
            lamina[i, 7] = int(l[7]*phi_to_n_max(cyl, lamina[i,4])) # defining n_2
    
    # if lam_type = 0 (i.e. SF laminate)
    elif not lam_type:
        
        for i, l in enumerate(lhs_in):
            lamina[i, 0] = round(l[0]*90) # defining alpha_1
            lamina[i, 1] = round(l[1]*90) # defining alpha_2
    
    return lamina

## Rapid tow sheared (RTS) laminates
# number of RTS variables: [pm phi_1 < T0_1 | T1_1 > n_1 , pm phi_2 < T0_2 | T1_2 > n_2]s
n_rts_var = 8

# defining number of samples for RTS laminates
n_rts_sam = 10*n_rts_var
# the number of samples is based on the 'levels' of each variable:
# # https://stats.stackexchange.com/questions/58201/how-to-determine-the-sample-size-of-a-latin-hypercube-sampling
# SMT use 10 * n dimensions. We have 8 dimensions, therefore 80 samples. Could change... Probably need to infill after initial 

from pyDOE2 import lhs

# call LHS from pyDOE2 for RTS laminates
lhs_rts  = lhs(n_rts_var, n_rts_sam)

## Straight fibre (SF) laminates
# number of SF variables: [pm alpha_1, pm alpha_2]s
n_sf_var = 2

# number of samples for SF laminates
n_sf_sam = 10*n_sf_var

# call LHS from pyDOE2 for SF laminates
lhs_sf = lhs(n_sf_var, n_sf_sam)

# cyl is a vector with [radius, length, thickness] in mm of the cylinder in question
# cyl is needed to define max periodicty if RTS laminate is chosen
cyl = [300, 1040, 1.05]

# transform unity lhs to laminate definition for RTS (lam_type = 1)
rts_lams = lhs_to_lamina(lhs_rts, cyl, 1)

# transform unity lhs to laminate definition for SF (lam_type = 0)
sf_lams = lhs_to_lamina(lhs_sf, cyl, 0)

# Run LHS samples through Abaqus


In [7]:
# Run LHS with Abaqus. For now, use dummy function where y (output) is the sum of the variables
def dummy_func(x_in):
    
    y_out = []
    
    for x in x_in:
        y_out.append(sum(x))
    
    return np.asarray(y_out)

y = dummy_func(rts_lams)

print(y)

[319. 289. 200. 374. 160. 210. 366. 255. 191. 302. 298. 182. 359. 279.
 198. 269. 273. 207. 185. 191. 301. 361. 178. 259. 328. 332. 177. 384.
 220. 241. 266. 309. 230. 211. 117. 137. 272. 307. 304. 282. 403. 283.
 316. 195. 164. 167. 264. 272. 195. 170. 190. 162. 267. 111. 241. 238.
 241. 296. 277. 268. 101. 239. 193. 267. 195. 310. 105. 198. 242. 119.
 255. 223. 334. 210. 216. 239. 299. 247. 327. 181.]


# Calculate convergence crit

In [8]:
from smt.surrogate_models import KRG

sm = KRG(theta0=[1e-1])
sm.set_training_values(rts_lams, y)
sm.train()

rng = np.random.default_rng()
n_test = 5
x_test = lhs_to_lamina(rng.random((n_test,8)),cyl,1)

print(x_test)

y_pred = sm.predict_values(x_test)

print(y_pred)

y_true = dummy_func(x_test)

print(y_true)

# estimated variance
# s2 = sm.predict_variances(x)
# derivative according to the first variable
# dydx = sm.predict_derivatives(xt, 0)
# fig, axs = plt.subplots(1)

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 80
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.3723986
[[90.  9. 18.  3.  0. 17. 11.  3.]
 [ 0. 65. 41.  0.  0. 40. 11.  0.]
 [ 0. 14. 32.  2.  0. 61. 16.  1.]
 [ 0. 25. 34.  7. 90. 25. 31. 10.]
 [ 0. 53. 44.  9.  0. 43. 37.  6.]]
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 5
   
   Predicting ...
   Predicting - done. Time (sec):  0.0000000
   
   Prediction time/pt. (sec) :  0.0000000
   
[[151.00013421]
 [157.00020191]
 [126.00004046]
 [222.00011038]
 [192.00007484]]
[151. 157. 126. 222. 192.]
